In [1]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'


In [2]:
# connect kafka with spark with simple word count example
# run on a terminal after starting kafka
#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic test
#     kafka-console-producer --bootstrap-server localhost:9092 --topic test  


# here we implement windowed word count, the word count would reset every minute, we fix  5 minute window

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic word-counts-5min 
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  word-counts-5min  --from-beginning  


In [3]:
import findspark
findspark.init()

In [4]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaBasic").getOrCreate()

22/05/25 00:48:14 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/25 00:48:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3f688fc2-24d7-4789-9e9b-841096968bf7;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 2195ms :: artifacts dl 29ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from centra

In [5]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1

kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "test")\
  .option("group.id", "wordcount-group")\
  .load()

In [6]:
# key and value are binary type, we need to CAST To STRING type
kafkaDf.printSchema()
# timestampType values
# CreateTime:  Timestamp relates to message creation time as set by a Kafka client/producer
# LogAppendTime : Timestamp relates to the time a message was appended to a Kafka log.
# UnknownType

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# Kafka value, key are in binary format
# we need to type cast to STRING, 
# we pick only value, timestamp as we need for 5 minute window
linesDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
linesDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
# THUMBLING WINDOW 
# Also known as NON OVERLAPPING WINDOW
# No duplicate data processing
# window("5 minutes") Thumbling Window [No Slide Time mentioned]

import pyspark.sql.functions as F
# split line into word list
# flatten word list into individual element as output, similar to flatMap

wordsDf = linesDf.select(F.explode(F.split(linesDf.value," ")).alias("word"), "timestamp" )
wordsDf.printSchema()
# apply 5 Minute Window
# groupBy can be useful to mention more columns, we are 1 minute window as another group by
# within minute, the count start from 0, goes on based on number words
# Thumbling Window
wordCountsDf = wordsDf.groupBy("word", F.window(wordsDf.timestamp, "5 minutes")).count()
wordCountsDf.printSchema()

root
 |-- word: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

root
 |-- word: string (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- count: long (nullable = false)



In [9]:
echoOnconsole = wordCountsDf\
                .writeStream\
                .outputMode("complete")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

22/05/25 00:51:33 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5adfc41e-7e82-424b-ad2e-5aa0075ac029. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/25 00:51:36 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9686d3c0-7115-463a-8f73-475aa3907251-1308792722-driver-0-1, groupId=spark-kafka-source-9686d3c0-7115-463a-8f73-475aa3907251-1308792722-driver-0] Error while fetching metadata with correlation id 2 : {test=LEADER_NOT_AVAILABLE}
22/05/25 00:51:36 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-9686d3c0-7115-463a-8f73-475aa3907251-1308792722-driver-0-1, groupId=spark-kafka-source-9686d3c0-7115-463a-8f73-475aa3907251-1308792722-driver-0] Error while fetching metadata with correlation id 6 : {test=LEADER_NOT_A

In [10]:
# now publish the word count result (word, count columns) to kafka topic "word-counts", publish as json format
# {"word": "kafka", "count": 8}

# F is alias for all functions, we can access col by F.col 
import pyspark.sql.functions as F

# convert all the columns into json
# * represent all columns ie word, count, 
# struct create a structure around word, count columns
# to json convert structure to column
# value is Kafka value part of message
wordCountsToKafkaDf = wordCountsDf\
                    .selectExpr("to_json(struct(*)) as value")

wordCountsToKafkaDf.printSchema()

root
 |-- value: string (nullable = true)



-------------------------------------------
Batch: 0
-------------------------------------------
+----+------+-----+
|word|window|count|
+----+------+-----+
+----+------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+------+-----+
|word|window|count|
+----+------+-----+
+----+------+-----+



In [11]:
# checkpointLocation is for storing local state, for system restart, system failure in between
# ensure to run kafka console consumer for topic "word-counts-5min, commands are present in top of file
wordCountsToKafkaDf.writeStream.format("kafka")\
                    .outputMode("complete")\
                     .option("kafka.bootstrap.servers", "localhost:9092")\
                    .option("topic", "word-counts-5min")\
                    .option("checkpointLocation", "file:///tmp/spark22")\
                    .start()
                    

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+--------------------+-----+
| word|              window|count|
+-----+--------------------+-----+
|kafka|{2022-05-25 00:55...|    1|
|spark|{2022-05-25 00:55...|    1|
+-----+--------------------+-----+



22/05/25 00:55:11 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 1 : {word-counts-5min=LEADER_NOT_AVAILABLE}
22/05/25 00:55:12 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 3 : {word-counts-5min=LEADER_NOT_AVAILABLE}


-------------------------------------------
Batch: 3
-------------------------------------------
+-----+--------------------+-----+
| word|              window|count|
+-----+--------------------+-----+
|kafka|{2022-05-25 00:55...|    2|
|spark|{2022-05-25 00:55...|    2|
+-----+--------------------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+------+--------------------+-----+
|  word|              window|count|
+------+--------------------+-----+
| kafka|{2022-05-25 00:55...|    2|
|apache|{2022-05-25 00:55...|    1|
| spark|{2022-05-25 00:55...|    2|
+------+--------------------+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+------+--------------------+-----+
|  word|              window|count|
+------+--------------------+-----+
| kafka|{2022-05-25 00:55...|    2|
|apache|{2022-05-25 00:55...|    1|
| spark|{2022-05-25 00:55...|    3|
+------+--------------------+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+------+--------------------+-----+
|  word|              window|count|
+------+--------------------+-----+
| kafka|{2022-05-25 00:55...|    2|
|apache|{2022-05-25 00:55...|    2|
| spark|{2022-05-25 00:55...|    3|
+------+--------------------+-----+



-------------------------------------------
Batch: 7
-------------------------------------------
+------+--------------------+-----+
|  word|              window|count|
+------+--------------------+-----+
| kafka|{2022-05-25 00:55...|    3|
|apache|{2022-05-25 00:55...|    3|
| spark|{2022-05-25 00:55...|    4|
+------+--------------------+-----+



-------------------------------------------
Batch: 8
-------------------------------------------
+------+--------------------+-----+
|  word|              window|count|
+------+--------------------+-----+
| kafka|{2022-05-25 00:55...|    3|
|apache|{2022-05-25 00:55...|    3|
| spark|{2022-05-25 00:55...|    4|
| kafka|{2022-05-25 01:00...|    1|
|apache|{2022-05-25 01:00...|    1|
| spark|{2022-05-25 01:00...|    1|
+------+--------------------+-----+



-------------------------------------------
Batch: 9
-------------------------------------------
+------+--------------------+-----+
|  word|              window|count|
+------+--------------------+-----+
|  java|{2022-05-25 01:00...|    1|
| kafka|{2022-05-25 00:55...|    3|
|apache|{2022-05-25 00:55...|    3|
| spark|{2022-05-25 00:55...|    4|
| kafka|{2022-05-25 01:00...|    2|
|apache|{2022-05-25 01:00...|    1|
| spark|{2022-05-25 01:00...|    2|
+------+--------------------+-----+

